<a href="https://colab.research.google.com/github/rdsiese/MANDES/blob/main/Latin_Hypercube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Latin Hypercube Sampling (LHS)

When sampling from a distribution, computers sample from a Uniform(0,1) that represents the vertical axis $[0,1]$ of the cumulative probability distribution. They then take the inverse cumulative distribution ($\texttt{ppf}$ in $\texttt{scipy.stats}$) to obtain the values of the variable. The sampling can be random or stratified, which is more regular.

Stratified sampling can be implemented with different packages. We will use the *Design Of Experiments* package $\texttt{pyDOE2}$.  

In [ ]:
!pip install pyDOE2

From this package we will use the *Latin Hypercube Sampling* function $\texttt{lhs}$.

In [ ]:
import numpy as np
from scipy.stats import norm
from pyDOE2 import lhs
import matplotlib.pyplot as plt

To show the difference between random (regular) sampling and LHS, let's generate a sample of a $N(0,1)$ distribution with both methods. The following code uses random sampling.

In [ ]:
n = 1000
z=np.random.normal(0,1,n)

plt.figure(figsize=(5,3))
plt.title('Random Sampling from a N(0,1)')
plt.hist(z, bins=50, edgecolor='white');

We now do the same operation, but with latin hypercube sampling.

In [ ]:
n = 1000
lhs_samples = lhs(1, samples=n)
z = norm.ppf(lhs_samples)

plt.figure(figsize=(5,3))
plt.title('Lating Hypercube Sampling from a N(0,1)')
plt.hist(z, bins=50, edgecolor='white');

## Option Pricing with LHS

To price the calls, we will define two functions, one for each method. The code will be the same within each function, except the sampling from the $N(0,1)$ distributions.

In [ ]:
def RS_call(S0,K,r,sigma,T,n):              # RS = Random Sampling
    payoff=np.zeros(n)
    S=np.zeros(n)
    z=np.random.normal(0,1,n)
    S=S0*np.exp( (r-sigma**2/2)*T + sigma*np.sqrt(T)*z )
    payoff=np.maximum(S-K,0)
    call = np.exp(-r*T)*np.mean(payoff)
    return call

In [ ]:
def LH_call(S0,K,r,sigma,T,n):              # LH = Latin Hypercube Sampling
    payoff=np.zeros(n)
    S=np.zeros(n)
    lhs_samples = lhs(1, samples=n)
    z = norm.ppf(lhs_samples)
    S=S0*np.exp( (r-sigma**2/2)*T + sigma*np.sqrt(T)*z )
    payoff=np.maximum(S-K,0)
    call = np.exp(-r*T)*np.mean(payoff)
    return call

We introduce the values of the parameters and compute the prices with each method, running only 1,000 scenarios. We compare the results with the Black-Scholes price.

In [ ]:
S0=100; K=100; r=0.05; sigma=0.2; T=1

In [ ]:
n=1000

RS_price = RS_call(S0, K, r, sigma, T, n)
LH_price = LH_call(S0, K, r, sigma, T, n)

print(f'RS_price:  {RS_price:<12.5f} n: {n:<10}')
print(f'LH_price:  {LH_price:<12.5f}')
print('price_BS:  10.45058')

Note how with **only 1,000 scenarios** the LHS option price is already quite close to the Black-Scholes price. Let's compute the prices with larger values of $n$.

In [ ]:
n=10000

RS_price = RS_call(S0, K, r, sigma, T, n)
LH_price = LH_call(S0, K, r, sigma, T, n)

print(f'RS_price:  {RS_price:<12.5f} n: {n:<10}')
print(f'LH_price:  {LH_price:<12.5f}')
print('price_BS:  10.45058')

In [ ]:
n=50000

RS_price = RS_call(S0, K, r, sigma, T, n)
LH_price = LH_call(S0, K, r, sigma, T, n)

print(f'RS_price:  {RS_price:<12.5f} n: {n:<10}')
print(f'LH_price:  {LH_price:<12.5f}')
print('price_BS:  10.45058')